In [1]:
import sys

sys.path.append("..")


In [2]:
from aurelio_sdk import AsyncAurelioClient
import os

client = AsyncAurelioClient(debug=True, api_key=os.environ["AURELIO_API_KEY_STAGING"], base_url=os.environ["BASE_URL_STAGING"])
print(client.base_url)


https://staging.api.aurelio.ai


# Extract text from PDFs and video MP4 files and urls


### From file


In [ ]:
# PDF

from aurelio_sdk import ExtractResponse

# file_path = "data/pdf/adaptive_semantic_search.pdf"
file_path = "data/pdf/test_pdf.pdf"

# NOTE: wait=-1 means wait till completion, polling_interval=5 means polling every 5 seconds for status till completion

response_pdf_file: ExtractResponse = await client.extract_file(
    file_path=file_path, quality="low", chunk=False, wait=-1, polling_interval=5
)

response_pdf_file

In [3]:
# Video
from aurelio_sdk import ExtractResponse

# From a local file
file_path = "data/video/how_to_overcome_our_mistakes.mp4"

response_video_file: ExtractResponse = await client.extract_file(
    file_path=file_path, quality="high", chunk=True, wait=-1, polling_interval=15
)

response_video_file


[AurelioSDK] [DEBUG] -- 2024-11-19 12:17:11 - at client_async.py:257 in _file_stream(): Reading chunk 1, chunk_size: 40.00 MB, total size: 7.88 MB
[AurelioSDK] [DEBUG] -- 2024-11-19 12:17:14 - at client_async.py:289 in extract_file(): Calling API
[AurelioSDK] [DEBUG] -- 2024-11-19 12:17:14 - at client_async.py:671 in wait_for(): Starting polling for document completion: doc_51ec8569-64a6-4ac8-b0b2-750c887686ac
[AurelioSDK] [DEBUG] -- 2024-11-19 12:17:30 - at client_async.py:691 in wait_for(): Polling document doc_51ec8569-64a6-4ac8-b0b2-750c887686ac: status=TaskStatus.pending
[AurelioSDK] [DEBUG] -- 2024-11-19 12:17:45 - at client_async.py:691 in wait_for(): Polling document doc_51ec8569-64a6-4ac8-b0b2-750c887686ac: status=TaskStatus.completed


ExtractResponse(status=<TaskStatus.completed: 'completed'>, usage=Usage(tokens=835, pages=None, seconds=291), message=None, processing_options=ExtractProcessingOptions(chunk=True, quality=<ProcessingQuality.high: 'high'>), document=ResponseDocument(id='doc_51ec8569-64a6-4ac8-b0b2-750c887686ac', content=" enrolled in introductory and advanced French courses. These students completed a questionnaire asking what kind of teacher they preferred, one who emphasized their strength and successes, or one who highlighted their mistakes and corrected their weaknesses. In general, responses showed that while beginner students sought positive reinforcement, advanced students were more eager for critical feedback. Researchers have theorized a handful of explanations for these results. Having just started out, beginners are still determining if they enjoy learning French, and if they want to continue studying.  the Roon study, participants in the failure group indicated much lower levels of self-conf

### From URL


In [ ]:
# From URL PDF
from aurelio_sdk import ExtractResponse

# From URL
url = "https://arxiv.org/pdf/2408.15291"
response_pdf_url: ExtractResponse = await client.extract_url(
    url=url, quality="low", chunk=True, wait=-1, polling_interval=5
)
response_pdf_url

In [ ]:

response_pdf_url.document.chunks[0].num_tokens

In [ ]:
# From URL Video
from aurelio_sdk import ExtractResponse

# From URL
url = "https://storage.googleapis.com/gtv-videos-bucket/sample/ForBiggerMeltdowns.mp4"
response_video_url: ExtractResponse = await client.extract_url(
    url=url, quality="low", chunk=True, wait=-1
)

response_video_url

In [ ]:
video_response_dict = response_video_url.model_dump()
video_response_dict["document"]["chunks"][0]["metadata"]["start_time"]


# Get document status and handle timeouts


In [ ]:
from aurelio_sdk import ExtractResponse

# From a local file
file_path = "data/pdf/adaptive_semantic_search.pdf"

response_pdf_file: ExtractResponse = await client.extract_file(
    file_path=file_path, quality="high", chunk=True, wait=10)

# Get document status and document id
print("Status:", response_pdf_file.status)
print("Document ID:", response_pdf_file.document.id)

In [ ]:
# Get document status and response
document_response: ExtractResponse = await client.get_document(
    document_id=response_pdf_file.document.id, timeout=1
)

print("Status:", document_response.status)

In [ ]:
# Use a pre-built function, which helps to avoid long hanging requests (Recommended)
document_response = await client.wait_for(
    document_id=response_pdf_file.document.id, wait=300
)

In [ ]:
document_response